In [ ]:
# ANMOL Uncomment the first time to install
#!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
# UNINSTALL PYCOCOTOOLS IF YOU ALREADY HAVE IT INSTALLED
#!pip uninstall pycocotools --yes
import sys
import os
cs231_repo_path = "/mount/ws/code_devel/cs231n-project"
sys.path.append(cs231_repo_path)
#detectron_path = os.path.join(cs231_repo_path, "detectron2_code")
#!cd {detectron_path} && python -m pip install -e .

In [ ]:
#!pip uninstall detectron2 --yes

In [ ]:
import collections
import torch
import json
import os
import cv2
import random
import gc
import pycocotools
import torch.nn.functional as F

import numpy as np 
import pandas as pd 
from tqdm import tqdm
import matplotlib.pyplot as plt
import PIL
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader

from pathlib import Path

# import some common detectron2 utilities
from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# ANMOL: ADD PATH TO DATASET
# DIRECTORY SHOULD HAVE ['train', 'sample_submission.csv', 'train.csv', 'label_descriptions.json', 'test']
dataDir = "/mount/data"

In [ ]:
# ANMOL:This Cell is needed to get names of object and attribute classes
# Dictionaries: idx_to_did and did_to_idx are important for Training Attr Classifier
with open(os.path.join(dataDir, 'label_descriptions.json'), 'r') as file:
    label_description = json.load(file)
#label_description
n_classes = len(label_description['categories'])
n_attributes = len(label_description['attributes'])
print(F"Classes: {n_classes}, Attributes: {n_attributes}")
categories_df = pd.DataFrame(label_description['categories'])
attributes_df = pd.DataFrame(label_description['attributes'])
idx_to_did = {}
did_to_idx = {}
for idx, entry in attributes_df.iterrows():
    idx_to_did[idx] = entry['id']
    did_to_idx[entry['id']] = idx

In [ ]:
# ANMOL: This is processed version of train.csv. This combines all the segments, classes, boxes and attributes into
# one single list entry which makes it a bit easier to process. I have uploaded 2 json on cloud (one with 2000 
# images and other with 5000 images)
wsDir = "/mount/ws/attr_cls_test"
fashion_dict = []
with open(os.path.join(wsDir, 'fashion_training.json'), 'r') as file:
    fashion_json = json.load(file)
fashion_dict = fashion_json["train"]
# Just sanity check
print(fashion_dict[0].keys())
print(fashion_dict[0]['file_name'])

In [ ]:
# This block has all the functionality to read process the data: Masks, Bounding Boxes, Attributes 
# that is consumed by dataloader

import math, numpy

def rle_decode_string(string, h, w):
        mask = np.full(h*w, 0, dtype=np.uint8)
        annotation = [int(x) for x in string.split(' ')]
        for i, start_pixel in enumerate(annotation[::2]):
            mask[start_pixel: start_pixel+annotation[2*i+1]] = 1
        mask = mask.reshape((h, w), order='F')
        return mask

def rle2bbox(rle, shape):
    '''
        Get a bbox from a mask which is required for Detectron 2 dataset
        rle: run-length encoded image mask, as string
        shape: (height, width) of image on which RLE was produced
        Returns (x0, y0, x1, y1) tuple describing the bounding box of the rle mask
        
        Note on image vs np.array dimensions:
        
            np.array implies the `[y, x]` indexing order in terms of image dimensions,
            so the variable on `shape[0]` is `y`, and the variable on the `shape[1]` is `x`,
            hence the result would be correct (x0,y0,x1,y1) in terms of image dimensions
            for RLE-encoded indices of np.array (which are produced by widely used kernels
            and are used in most kaggle competitions datasets)
    '''
        
    a = np.fromiter(rle.split(), dtype=np.uint)
    a = a.reshape((-1, 2))  # an array of (start, length) pairs
    a[:,0] -= 1  # `start` is 1-indexed

    y0 = a[:,0] % shape[0]
    y1 = y0 + a[:,1]
    if np.any(y1 > shape[0]):
        # got `y` overrun, meaning that there are a pixels in mask on 0 and shape[0] position
        y0 = 0
        y1 = shape[0]
    else:
        y0 = np.min(y0)
        y1 = np.max(y1)

    x0 = a[:,0] // shape[0]
    x1 = (a[:,0] + a[:,1]) // shape[0]
    x0 = np.min(x0)
    x1 = np.max(x1)

    if x1 > shape[1]:
        # just went out of the image dimensions
        raise ValueError("invalid RLE or image dimensions: x1=%d > shape[1]=%d" % (
            x1, shape[1]
        ))

    return int(x0), int(y0), int(x1), int(y1)
    
def get_attr(attr_str, max_class, labels_length):
    if attr_str == '' or (isinstance(attr_str, float) and  math.isnan(attr_str)):
        return [max_class for i in range(labels_length)]
    else:
        attr_list = [did_to_idx[int(x)] for x in attr_str.split(",")]
        fillers = [max_class for i in range(labels_length - len(attr_list))]
        attr_list.extend(fillers)
        return attr_list




In [ ]:
# ANMOL: MAKE SURE YOU dataDir variable is set above dataDir = "/mount/data/train"

# https://detectron2.readthedocs.io/tutorials/datasets.html
# https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5
import os
from detectron2.structures import BoxMode
import pycocotools
train_path = "/mount/data/train"

def get_fashion_dict(df):
    
    dataset_dicts = []
    path_name = train_path
    
    for idx, df_entry in enumerate(df):
        
        record = {}
        
        # Convert to int otherwise evaluation will throw an error
        record['height'] = int(df_entry['height'])
        record['width'] = int(df_entry['width'])
        
        record['file_name'] = os.path.join(path_name, df_entry['file_name']+".jpg")
        record['image_id'] = idx
        
        objs = []
        for mask_entry, class_entry, attr_entry in zip(df_entry['masks'], df_entry['labels'], df_entry['attributes']):
            #print(mask_entry)
            mask = rle_decode_string(mask_entry, record['height'], record['width'])
            # opencv 4.2+
            # Transform the mask from binary to polygon format
            contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                    cv2.CHAIN_APPROX_SIMPLE)
            
            # opencv 3.2
            # mask_new, contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
            #                                            cv2.CHAIN_APPROX_SIMPLE)
            segmentation = []

            for contour in contours:
                contour = contour.flatten().tolist()
                # segmentation.append(contour)
                if len(contour) > 4:
                    segmentation.append(contour)  
            obj = {
                'bbox': list(rle2bbox(mask_entry, (record['height'], record['width']))),
                'bbox_mode': BoxMode.XYXY_ABS,
                'category_id': class_entry,
                'attributes': get_attr(attr_entry, 294, 14),
                'segmentation': segmentation,
                'iscrowd': 0
            }
            objs.append(obj)
        record['annotations'] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
# ANMOL: THIS IS JUST FOR Visualization later after the network is trained
d2_ds = get_fashion_dict(fashion_dict[0:10])
print(len(d2_ds))

In [ ]:
# ANMOL: This Structure is used to store Attributes. getitem, get_tensor are important

from typing import Iterator, List, Tuple, Union
import torch.nn.functional as F

# Base Attribute holder
class Attributes:
    """
    This structure stores a list of attributes as a Nx14 torch.
    Int training dataset, there were a maximum of 14 attributes per instance
    https://detectron2.readthedocs.io/_modules/detectron2/structures/boxes.html#Boxes
    It behaves like a Tensor
    (support indexing, `to(device)`, `.device`, `non empty`, and iteration over all attributes)
    """
    
    def __init__(self, tensor: torch.Tensor):
        """
        Args:
            tensor (Tensor[float]): a Nx14 matrix.  Each row is [attribute_1, attribute_2, ...].
        """
        device = tensor.device if isinstance(tensor, torch.Tensor) else torch.device("cpu")
        tensor = torch.as_tensor(tensor, dtype=torch.int64, device=device)
        assert tensor.dim() == 2, tensor.size()

        self.tensor = tensor


    def __getitem__(self, item: Union[int, slice, torch.BoolTensor]) -> "Attributes":
        """
        Returns:
            Attributes: Create a new :class:`Attributes` by indexing.
        The following usage are allowed:
        1. `new_attributes = attributes[3]`: return a `Attributes` which contains only one Attribute.
        2. `new_attributes = attributes[2:10]`: return a slice of attributes.
        3. `new_attributes = attributes[vector]`, where vector is a torch.BoolTensor
           with `length = len(attributes)`. Nonzero elements in the vector will be selected.
        Note that the returned Attributes might share storage with this Attributes,
        subject to Pytorch's indexing semantics.
        """
        
        b = self.tensor[item]
        assert b.dim() == 2, "Indexing on Attributes with {} failed to return a matrix!".format(item)
        return Attributes(b)

    def __len__(self) -> int:
        return self.tensor.shape[0]
    
    def to(self, device: str) -> "Attributes":
        return Attributes(self.tensor.to(device))


    def nonempty(self, threshold: float = 0.0) -> torch.Tensor:
        """
        Find attributes that are non-empty.
        An attribute is considered empty if its first attribute in the list is 294.
        Returns:
            Tensor:
                a binary vector which represents whether each attribute is empty
                (False) or non-empty (True).
        """
        attributes = self.tensor
        first_attr = attributes[:, 0]
        keep = (first_attr != 294)
        return keep

    def __repr__(self) -> str:
        return "Attributes(" + str(self.tensor) + ")"

    def get_shape(self) -> Tuple[int, int]:
        return self.tensor.shape[0], self.tensor.shape[1]
    
    def get_tensor(self) -> torch.Tensor:
        return self.tensor
    
    def remove_padding(self, attribute):
        pass

    @property
    def device(self) -> torch.device:
        return self.tensor.device

    def __iter__(self) -> Iterator[torch.Tensor]:
        """
        Yield attributes as a Tensor of shape (14,) at a time.
        """
        yield from self.tensor

In [ ]:
# ANMOL: This is used to add gt_attributes to dataset mapper.

import copy
from detectron2.data import build_detection_train_loader, build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils


class DatasetMapper:
    """
    A callable which takes a dataset dict in Detectron2 Dataset format,
    and map it into a format used by the model.
    Derived From: https://github.com/facebookresearch/detectron2/blob/master/detectron2/data/dataset_mapper.py
    Just Add Attributes field to instances  
    """

    def __init__(self, cfg, is_train=True):

        self.crop_gen = None

        self.tfm_gens = utils.build_transform_gen(cfg, is_train)

        # fmt: off
        self.img_format     = cfg.INPUT.FORMAT
        self.mask_on        = cfg.MODEL.MASK_ON
        self.mask_format    = cfg.INPUT.MASK_FORMAT
        self.keypoint_on    = cfg.MODEL.KEYPOINT_ON
        self.load_proposals = cfg.MODEL.LOAD_PROPOSALS
        # fmt: on
        self.keypoint_hflip_indices = None

        self.is_train = is_train

    def __call__(self, dataset_dict):
        """
        Args:
            dataset_dict (dict): Metadata of one image, in Detectron2 Dataset format.

        Returns:
            dict: a format that builtin models in detectron2 accept
        """
        dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
        # USER: Write your own image loading if it's not from a file
        image = utils.read_image(dataset_dict["file_name"], format=self.img_format)
        utils.check_image_size(dataset_dict, image)

        if "annotations" not in dataset_dict:
            image, transforms = T.apply_transform_gens(
                ([self.crop_gen] if self.crop_gen else []) + self.tfm_gens, image
            )
        else:
            # Crop around an instance if there are instances in the image.
            # USER: Remove if you don't use cropping
            image, transforms = T.apply_transform_gens(self.tfm_gens, image)

        image_shape = image.shape[:2]  # h, w

        # Pytorch's dataloader is efficient on torch.Tensor due to shared-memory,
        # but not efficient on large generic data structures due to the use of pickle & mp.Queue.
        # Therefore it's important to use torch.Tensor.
        dataset_dict["image"] = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)))


        if not self.is_train:
            # USER: Modify this if you want to keep them for some reason.
            dataset_dict.pop("annotations", None)
            dataset_dict.pop("sem_seg_file_name", None)
            return dataset_dict

        if "annotations" in dataset_dict:
            # USER: Modify this if you want to keep them for some reason.
            for anno in dataset_dict["annotations"]:
                if not self.mask_on:
                    anno.pop("segmentation", None)
                if not self.keypoint_on:
                    anno.pop("keypoints", None)

            # USER: Implement additional transformations if you have other types of data
            annos = [
                utils.transform_instance_annotations(
                    obj, transforms, image_shape, keypoint_hflip_indices=self.keypoint_hflip_indices
                )
                for obj in dataset_dict.pop("annotations")
                if obj.get("iscrowd", 0) == 0
            ]
            instances = utils.annotations_to_instances(
                annos, image_shape, mask_format=self.mask_format
            )
            # Create a tight bounding box from masks, useful when image is cropped
            if self.crop_gen and instances.has("gt_masks"):
                instances.gt_boxes = instances.gt_masks.get_bounding_boxes()           

            if len(annos) and 'attributes' in annos[0]:
    
                # get a list of list of attributes
                gt_attributes = [x['attributes'] for x in annos]
                instances.gt_attributes = Attributes(gt_attributes)
                
            dataset_dict["instances"] = utils.filter_empty_instances(instances)

        return dataset_dict

In [ ]:
# ANMOL: For our on Mapper, we need to define train and test loaders
from detectron2.engine import DefaultTrainer

class TrendTrainer(DefaultTrainer):
    'A customized version of DefaultTrainer. We add the mapping `DatasetMapper` to the dataloader.'
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg))
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name, mapper=DatasetMapper(cfg))

In [ ]:
# Get a sample of the training data to run experiments
# You may set this number based on your dataset size

df_copy_train = fashion_dict[:40000].copy()
df_copy_test = fashion_dict[40000:42000].copy()
print(len(df_copy_train))
print(len(df_copy_test))

In [ ]:
#df_copy_test = fashion_dict[40000:40500].copy()
#print(len(df_copy_test))

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# Register the train set metadata
for d in ['train']:
    DatasetCatalog.register('2sample_fashion_' + d, lambda d=df_copy_train: get_fashion_dict(d))
    MetadataCatalog.get("2sample_fashion_" + d).set(thing_classes=list(categories_df.name))
fashion_metadata = MetadataCatalog.get("2sample_fashion_train")

In [ ]:
# Register the test and set metadata
for d in ['test']:
    DatasetCatalog.register('2sample_fashion_' + d, lambda d=df_copy_test: get_fashion_dict(d))
    MetadataCatalog.get("2sample_fashion_" + d).set(thing_classes=list(categories_df.name))
fashion_metadata = MetadataCatalog.get("2sample_fashion_test")

In [ ]:
#trainer = DefaultTrainer(cfg) 
#from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
#add_trendrcnn_config(cfg)
#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("2sample_fashion_train",)
cfg.DATASETS.TEST = ()
#cfg.DATALOADER.NUM_WORKERS = 1
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
#cfg.MODEL.WEIGHTS = os.path.join("/mount/ws/code_devel/6_2_2020/experiments/rn50_fpn_m1_a0_i1_im5k_lr00025_bs4_mit5k", "model_final.pth")
#cfg.MODEL.WEIGHTS = os.path.join("/mount/ws/code_devel/6_2_2020/experiments/x101_m1_a0_i1_im5k_lr00025_bs4_mit5k_noAll0_focal_loss/output", "model_final.pth")
# ATTRIBUTE CLASSIFIER RELATED CONFIGS: 
cfg.MODEL.ROI_HEADS.NAME = "TrendRCNNROIHeads" # This is needed to running FPN, DC5, X_101 type networks
#cfg.MODEL.ROI_HEADS.NAME = "TrendRes5ROIHeads" # This is needed to run C4 kind of Network
# Number of attribute Classes DONT CHANGE
cfg.MODEL.ROI_HEADS.NUM_ATTR_CLASSES = 295
# Number of Max Attribute Predictions: DONT CHANGE
cfg.MODEL.ROI_HEADS.MAX_ATTR_PRED = 14
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 46  # 46 classes in iMaterialist

# These Options modify the Attr Classifier
# ATTR_CLS_MODE=0: 1 Linear Layer
# ATTR_CLS_MODE=1: Linear->RELU->Linear
cfg.MODEL.ROI_HEADS.ATTR_CLS_MODE=1
# Decides if Attr Classifier is agnostic to Object Class
# If 1, then final Linear Layer Output = 295
# if 0, the final Linear Layer Output = object classes (46) * attr_classes (295)
cfg.MODEL.ROI_HEADS.ATTR_CLS_AGNOSTIC=0
# Decide if we want to ignore object with no Attribute Label in Loss or not
# if 0: It will take No Attributes as one of the attr class, account for its loss
cfg.MODEL.ROI_HEADS.IGN_NAN_ATTR_CLS =1
cfg.MODEL.ROI_HEADS.ATTR_SCORE_THRESH_TEST=0.4

##### Input #####
# Set a smaller image size than default to avoid memory problems

# Size of the smallest side of the image during training
cfg.INPUT.MIN_SIZE_TRAIN = (800,)
# Maximum size of the side of the image during training
cfg.INPUT.MAX_SIZE_TRAIN = 1333
# Size of the smallest side of the image during testing. Set to zero to disable resize in testing.
cfg.INPUT.MIN_SIZE_TEST = 800
# Maximum size of the side of the image during testing
cfg.INPUT.MAX_SIZE_TEST = 1333


cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
#cfg.SOLVER.CHECKPOINT_PERIOD = 25
cfg.SOLVER.MAX_ITER = 20000   
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512


In [ ]:
# Use this set you output directory name clearly to store checkpoints
#"rn50_fpn_m1_a0_i1_im5k_lr00025_bs4_mit5k_attrLossSum"
#"remove_all0attrloss_rn50_fpn_m1_a0_i1_im5k_lr00025_bs4_mit5k"
#cfg.OUTPUT_DIR="rn50_fpn_m1_a0_i1_im5k_lr00025_bs4_mit5k_focal_loss_retinanet"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = TrendTrainer(cfg)

In [ ]:
trainer.resume_or_load(resume=True)

In [ ]:
trainer.train()

In [ ]:
# INFERENCE ON TRAINED WEIGHTS

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set the testing threshold for this model

cfg.DATASETS.TEST = ('2sample_fashion_test',)
predictor = DefaultPredictor(cfg)

In [ ]:
# Eval Phase
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
#trainer = DefaultTrainer(cfg)
#trainer2 = DefaultTrainer(cfg)
#trainer2.resume_or_load(resume=False)
#trainer = DefaultTrainer(cfg)
from detectron2.modeling import build_model
model = build_model(cfg)
from detectron2.checkpoint import DetectionCheckpointer
file_path=os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
DetectionCheckpointer(model).load(file_path)
#print(model)
evaluator = COCOEvaluator("2sample_fashion_test", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "2sample_fashion_test")
inference_on_dataset(model, val_loader, evaluator)

In [ ]:
# PLOTTING THE OUTPUT IMAGE WITH BOXES AND MASKS

from detectron2.utils.visualizer import ColorMode
plt.figure(figsize=(20,20))
for d in d2_ds[0:1]:
    #random.sample(d2_ds, 1):/
    #print(d["file_name"])
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    #print(outputs)
    v = Visualizer(im[:, :, ::-1],
                   metadata=fashion_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(v.get_image()[:, :, ::-1])

In [ ]:
# ANMOL: I use this to manually analyze and compare ground truth and predicted attributes.
# Print outputs variable from the above cell.. 

for entry in d2_ds[0]['annotations']:
    #print(entry['category_id'])
    print(F"CLASS LABEL: {entry['category_id']}, NAME: {categories_df.loc[entry['category_id'], 'name']}")
    #print(categories_df.loc[31, 'name'])
    print(entry['attributes'])
    for attr in entry['attributes']:
        if attr != 294:
            print("-------", attributes_df.loc[attr, 'name'])
            
print("---------OUTPUT----------")
output_attr = [[204, 160, 294, 157, 159, 207, 158, 206, 205, 209, 213, 216, 214, 210],
        [294, 218, 223, 224, 219, 222, 225, 121, 221,  89, 220, 247, 259,  63],
        [294, 163, 162, 166, 164, 170, 173, 168,  47, 161,  36, 167, 200, 210],
        [294, 218, 223, 224, 222, 219, 220, 190,  62, 276, 175, 217,  41, 108],
        [204, 160, 157, 294, 159, 205, 207, 209, 206, 213, 158, 216, 214, 211],
        [294, 174, 175, 176, 177, 178, 162,  80,  22, 151, 138, 284, 263,  52],
        [248, 270, 269, 154, 230, 115, 136, 142,  36, 135, 143, 251, 128, 234]]
#output_attr = [[294,  56, 160, 277, 268, 227,  64, 168,  77,  95, 256,  12, 273, 156],
#               [160, 274, 294,  64, 168, 204, 283,  77, 287, 220, 149, 153, 108,  95],
#               [269, 195, 265,   1, 118, 165, 145,  38,  60, 220,  37, 188, 264, 115],
#               [294,  90, 274, 166, 268, 287,  89, 160, 283,  41, 206,  64, 143,  84]]
#output_class = [31, 31,  1, 31]
output_class = [31, 32, 28, 32, 31, 29,  6]
for class_id, entry in enumerate(output_attr):
    print("Class NAME: ", categories_df.loc[output_class[class_id], 'name'])
    for idx in entry:
        if idx == 294:
            continue
        print("----:", attributes_df.loc[idx, 'name'])